Device tree overlay

In [ ]:
from mercury import overlay
o = overlay("mercury")

Hardware identification

In [ ]:
from mercury import hwid

regid = hwid()

hex(regid.dna)

In [ ]:
del(regid)

Arbitrary signal generator

In [ ]:
import numpy as np
from mercury import asg

asg0 = asg(0)

x = np.linspace(-np.pi, np.pi, asg1.N, endpoint=False)
wave = np.sin(x)

asg0.amplitude = 1.0
asg0.offset    = 0
# waveform should be loaded before frequency and phase are calculated
# this provides the correct buffer size
asg0.waveform  = wave
asg0.frequency = 10000
asg0.phase     = 0
asg0.trigger_mask = 0x1
asg0.trigger()

In [ ]:
asg1.regset.cfg_mul = 1.2 * asg1.DWMr

In [ ]:
del(asg0)

Acquire (oscilloscope)